Notebook esame: 
In questo notebook mi occuperò dell' analisi degli spettri di assorbimento di due specie di molecole (CO2 e C2H2), ottenuti attraverso il metodo 'comb-locked frequency-swept synthesis' (CLFSS), il quale permette di avere simultanaemante un grande livello di precisione e di range di misura in tempi brevi.
Lo scopo di questo progetto è trovare un procedimento semi-automatico per distinguere tra loro i dati, secondo ad esempio il criterio di appartenenza alle bande, e fittarli in modo ottimale, cercando di eliminare eventuali errori, per poter ricavare parametri utili ad un'analisi più quantitativa.
Come primo passo importo le librerie necessarie.

In [1]:
# Import libraries
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
#import ipywidgets as widgets
#from ipywidgets import interact_manual
#from ipywidgets import interact
from scipy.signal import find_peaks
import spettro_module as sm

Ora carico i dati attraverso la funzione loadtxt della libreria numpy, osservando i dati è possibile individuare una baseline, la quale viene subito sottratta ai dati

In [2]:
# Dizionario dei dati
dati = {}
bline=[.005,2.5e-9]
# Costruisco i vettori
myList=['C2H2', 'CO2'] 
dati['C2H2']=np.loadtxt("ATLAS_2018/C2H2_Pband_10Torr.dat",delimiter="\t", skiprows=2, usecols=range(6))
dati['CO2']=np.loadtxt("ATLAS_2018/CO2_3THz_scan_CRDS__data_2e-2Torr.dat",delimiter="\t", skiprows=2,usecols=range(6))
#tolgo immediatamente la baseline
for i,name in enumerate(myList):
    dati[name][:,-1]=dati[name][:,-1]-bline[i]
    #moltiplico i dati perche` non e` possible utilizzare uno dei metodi altrimenti
    if name=='CO2':
        dati[name][:,-1]=dati[name][:,-1]*1e7


In [3]:
#prova plot dati
plt.close('all')

for name in myList:
    plt.figure()
    plt.plot(dati[name][:,0],dati[name][:,-1])
    plt.xlabel('Frequenza [THz]')
    plt.ylabel('Assorbimento')
    plt.title(name)
    plt.grid()


Ora come prima analisi è utile trovare la posizione e l'altezza dei massimi dei picchi, i quali saranno l'oggetto d'interesse per la distinzione delle varie bande. Inoltre questi dati fungono da starting point per i fit di ogni picco. 

In [4]:
plt.close('all')
#algoritmo di peaks
peaks={}
#scelta manuale dei parametri ottimali
hmin=[.02, 6e-2]
width=10
for i,name in enumerate(myList):
    #definisco il dizionario dei picchi
    peaks[name]=[dati[name][find_peaks(dati[name][:,-1],height=hmin[i], width=width)[0],0],dati[name][find_peaks(dati[name][:,-1],height=hmin[i], width=width)[0],-1]]
    plt.figure()
    plt.plot(dati[name][:,0],dati[name][:,-1])
    plt.plot(peaks[name][0],peaks[name][1], '.')
    plt.xlabel('Frequenza [THz]')
    plt.ylabel('Assorbimento')
    plt.title(name)
    plt.grid()


Ora procediamo all' analisi in due metodi, che chiameremo 'fit dei picchi' e 'fit delle bande'. Il primo consiste nell'analizzare con un fit i singoli picchi di assorbimento, mentre per il secondo si procede con l'analisi dei massimi dei picchi appartenenti alla stessa banda. Questi due metodi non sono paralleli, bensì complementari, poiché, sebbene eseguibili indipendentemente tra loro, solo combinati riusciamo a raggiungere lo scopo di ordinamento e fit ottimale dei dati. Il metodo fit delle bande, infatti, permette di ricostruire con buona precisione, le diverse bande presenti utilizzando soltanto i massimi, quindi molti meno dati. Una volta ricostruita la banda, possiamo quindi ordinare i picchi a seconda della loro appartenenza alle bande, e implementare i risultati dati dal primo metodo, i quali saranno più precisi poiché non soggetti all'errore dello strumento di misura, per verificare che il modello sia corretto.

Per analizzare i picchi è necessario per prima cosa isolarli, per fare ciò inizio a togliere tutti i dati negativi, successivamente, utilizzando i massimi dei picchi trovati nella cella precedente come contatori, per ogni picco trovo un xmin e xmax, ovvero inizio e fine del picco con il minimo/massimo della funzione 1/(x-x_picco). Isolato il picco posso procedere con il fit, ma prima noto che alcuni picchi sono sovrapposti, di conseguenza non è possibile distinguerli e la funzione di fit non convergerà. Per eliminare tale problema, ho deciso di escludere dal fit tali picchi, attraverso un if. Per una migliore convergenza del fit, infine, ho scelto di utilizzzare starting points e bounds: per i primi, data la funzione di voigt definita nel modulo, ho utilizzato l'area sottesa da ogni picco trovata in approssimazione di integrale discreto di Riemann, e la Full Width at Half Maximum moltiplicata per alcuni fattori numerici; per i secondi mi sono avvalso di un vettore di controllo (ctrlv) per valutare l'errore percentuale del fit sul guess, e aggiustare i bounds di conseguenza, attraverso più esecuzioni della cella. 

In [12]:
#fit dei picchi
plt.close('all')

#per i risultati del fit e gli errori
fit_res=[[],[]]
efit_res=[[],[]]
#defininsco il vettore di controllo, nel quale salvero` gli errori percentuali per aggiustare i bounds 
ctrlv=[[],[]]
for i,name in enumerate(myList):
    plt.figure()
    plt.grid()
    #per selezionare il picco, uso come estremi i dati piu` vicini al centro del picco che pero` siano negativi
    cond=(dati[name][:,-1]<=0)
    dati2=dati[name][:,0]*cond
    fit_res[i]=np.array([0. for i in range(len(peaks[name][0]))])
    efit_res[i]=np.copy(fit_res[i])
    plt.plot(dati[name][:,0],dati[name][:,-1],'k',linewidth=.1)
    for c,j in enumerate(peaks[name][0]):
        #perche` questa formula? voglio trovare il pt piu` vicino che sia sotto la bline(0) (altrimenti arrivo al picco), poi aggiungo +/-1 per trovarli solo positivi
        xmin=np.argmin(1/(dati2-j))+1
        xmax=np.argmax(1/(dati2-j))-1
        xfit=dati[name][xmin:xmax,0]
        yfit=dati[name][xmin:xmax,-1]
        #grazie all'if elimino i picchi sovrapposti, se trova un picco nell intervallo (xmin,xmax) non lo considera
        if c!=len(peaks[name][0])-1:
            if np.logical_and(peaks[name][0][c+1]>xfit[-1],peaks[name][0][c-1]<xfit[0]):
                
                FWHM=xfit[yfit>=max(yfit)/2][-1]-xfit[yfit>=max(yfit)/2][0]

                #per il fit utilizzo la funzione di voigt con starting points 
                p0=[np.dot(np.diff(xfit),yfit[:-1]),FWHM/(2*np.sqrt(2*np.log(2))),j,FWHM/2]

                fit,cov=curve_fit(sm.voigt,xfit,yfit,p0=p0,
                                  bounds=((p0[0]/2,p0[1]/2,p0[2]-.1,p0[3]*.09),(p0[0]*2,p0[1],p0[2]+.1,p0[3]*2)))
                #grazie a questo vettore controllo la bonta` del guess in fase di revisione dati
                ctrlv[i].append((p0-fit)/p0)
                #salvo i risultati del fit (basta solo il coefficente di normalizzazione), in un vettore
                fit_res[i][c]=fit[0]
                efit_res[i][c]=np.diag(cov)[0]

                plt.plot(xfit,yfit,'r',linewidth=1)
                plt.plot(xfit,sm.voigt(xfit,*fit),'--b',linewidth=.5)
                plt.plot(xfit,sm.voigt(xfit,*p0),'--g',linewidth=.5)
    plt.xlabel('Frequenza [THz]')
    plt.ylabel('Assorbimento')
    plt.title(name)
    plt.legend(['whole profile','selected data','fit curve','guess'])


Si può notare come il guess non sia molto preciso, per migliorare ciò si potrebbe fare uno studio più approfondito sulla teoria della funzione di voigt oppure usare i risultati del vettore di controllo per aggiustarli euristicamente. Noto che il fit sui dati di CO2 è stato possibile soltanto aumentando il valore dei dati in y, altrimenti i parametri del fit sarebbero stati troppo bassi.

Illustriamo ora il secondo metodo, ovvero il fit delle bande. Per fittare il modello, ovvero una distribuzione di Maxwell-Boltzmann, occorre innanzitutto distinguere le bande, poiché per ora non lo sono. Per fare ciò osservo che dal plot dei massimi dei picchi, il profilo che ne risulta è a sua volta un profilo a picchi, quindi posso riutilizzare la funzione find_peaks per distinguere la banda più alta dal resto dei dati, utilizzando i parametri distance e hmin, scelti dall'utente, per selezionare solo i dati desiderati. Noto inoltre che questo metodo permette anche di dare una stima del centro della banda poiché esso è punto di minimo relativo 

In [6]:
#provo a plottare i massimi dei massimi e vedere se riconosce le bande
plt.close('all')
peaks2={}
#centri delle bande
gmu=[0,0]
#parametri manuali di find peaks
hmin=[.15, 2e-1]
dist=[7,2] 
for i,name in enumerate(myList):
    #in maniera del tutto simile a prima definisco i vettori dei massimi appartenenti alle bande
    peaks2[name]=[peaks[name][0][find_peaks(peaks[name][1],height=hmin[i],distance=dist[i])[0]],peaks[name][1][find_peaks(peaks[name][1],height=hmin[i],distance=dist[i])[0]]]
    #e` anche possibile dare cosi` una stima del centro delle bande
    gmu[i]=peaks2[name][0][find_peaks(-peaks2[name][1])[0]]
    
    plt.figure()
    plt.plot(peaks[name][0],peaks[name][1], 'r-*')
    plt.plot(peaks2[name][0],peaks2[name][1], 'b.')
    plt.plot(gmu[i],0,'g.')
    plt.xlabel('Frequenza [THz]')
    plt.ylabel('Assorbimento')
    plt.title(name)
    plt.grid()


Una volta distinta la banda superiore dal resto, noto che molti punti che dovrebbero appertenervi mancano. Questo perchè il metodo precedente riesce a distinguerli soltanto se rappresentano dei massimi, se si trovano in una zona in cui non sono presenti massimi di altre bande, questi non spiccheranno tra i dati, e quindi non verranno considerati. Per ovviare a ciò eseguo un fit preliminare di questa prima scrematura di bande, sempre utilizzando degli starting points trovati manipolando la funzione di riferimento, successivamente considero nuovamente tutti i massimi. Se questi distano dal modello meno di una certa ε, arbitraria ma in prima approssimazione uguale a circa  2,5 volte la deviazione standard dei dati selezionati dal modello, allora anche essi vengono considerati parte della banda. Ora che ho aggiunto alcuni dati, con un certo limite dato da una soglia in altezza (poiché al di sotto di essa vi sono troppe bande e quindi i dati sono troppo densi), eseguo nuovamente il fit, in modo da avere dei parametri migliori del modello, il quale di adatterà meglio ai dati.

In [11]:
#fit delle bande
plt.close('all')
kb=1.380649 *10**-23
#questi sono i parametri iniziali del centro di banda e espsilon dati a mano
#gmu=[196.8,190.29] 
#eps=[2e-1,3e-8]
gmu=[0,0]
eps=[0,0]
molt=[3,4]
#idea: ciclo while che diminuisce gradualmente eps ma lo tiene tale che almeno tutti i selected point siano nella ricostruzione

#dichiaro vettori degli starting points dei fit
vmax=[0,0]
norm=[0,0]

#la prima parte di codice e` uguale alla cella precedente, la quale esiste soltanto per illustrare il metodo, che viene implementato meglio qui
peaks2={}
hmin=[.15, 2e-1]
dist=[7,2]

#questo vettore serve a salvare le selezioni delle bande per l'utilizzo nell'ultima cella e il successivo per salvare i risultati del fit
bfin=[[[],[]],[[],[]]]
fit_mb=[[[],[]],[[],[]]]
efit_mb=[[[],[]],[[],[]]]

for i,name in enumerate(myList):
    #idea: potrei inserire anche un ciclo per fittare e distinguere piu` bande e non solo quella maggiore
    #la prima parte di codice e` uguale alla cella precedente, la quale esiste soltanto per illustrare il metodo, che viene implementato meglio qui
    peaks2[name]=[peaks[name][0][find_peaks(peaks[name][1],height=hmin[i],distance=dist[i])[0]],peaks[name][1][find_peaks(peaks[name][1],height=hmin[i],distance=dist[i])[0]]]
    gmu[i]=peaks2[name][0][find_peaks(-peaks2[name][1])[0]]
    #definisco la x del massimo della banda, serve a distinguere se la banda `e a dx o sx del centro
    b_max=np.argmax(peaks2[name][1])
    plt.figure()
    plt.plot(peaks[name][0],peaks[name][1],'k.',markersize=2)
    #questo for serve per poter fittare sia la banda dx che sx, siccome sono asimmetriche, e gli if selezionano i dati rispetto al centro, tranne nel caso C2H2 dove c'e` solo la sinistra
    for k in range(2):
        if peaks2[name][0][b_max]>gmu[i]:
            cond2=peaks2[name][0]>=gmu[i]
            cond3=peaks[name][0]>=gmu[i]
        else:
            cond2=peaks2[name][0]<=gmu[i]
            cond3=peaks[name][0]<=gmu[i]
        if np.logical_and(i!=0,k==1):
            cond2=np.logical_not(cond2)
            cond3=np.logical_not(cond3)
        xb=np.copy(peaks2[name][0][cond2])
        yb=np.copy(peaks2[name][1][cond2])
        vmax[i]=abs(xb[np.argmax(yb)]-gmu[i])
        #vmax e` possibile trovarlo anche conoscendo T e m, difatti se si vuole ricavare la temperatura bisogna conoscere m e il risultato del fit
        #come trovo la norma se non distinguo ancora i dati, e quindi non posso integrare? 
        #f(vmax)=C*... e inverto la relazione
        norm[i]=np.exp(1)*yb[np.argmax(yb)]*abs(vmax[i])*(np.pi**(1/2))/4
        #starting points
        p0=[vmax[i],norm[i],gmu[i]]
        fit,cov=curve_fit(sm.max_boltz,xb,yb,p0=p0)
        #si potrebbe computare eps partendo ad esempio dai residui o dallo scarto quadratico medio, ho scelto 2.5 volte la std perche` + grande
        eps[i]=molt[i]*np.sqrt(np.mean((yb-sm.max_boltz(xb,*fit))**2))
        #ricostruzione banda, se un punto dei massimi si avvicina a meno di eps, e` + alto di hmin e si trova nel giusto lato, viene accettato
        bcon=((abs(peaks[name][1]-sm.max_boltz(peaks[name][0],*fit)))<=eps[i])*cond3*(peaks[name][1]>=hmin[i])
        #se pongo come condizione che il picco lo abbia fittato anche con il vogit, ottengo una condizione per l'ultima parte del codice
        bfin[i][k]=bcon*(fit_res[i]!=0)

        fit2,cov2=curve_fit(sm.max_boltz,peaks[name][0][bcon],peaks[name][1][bcon],p0=p0)
        fit_mb[i][k]=fit2[0]
        efit_mb[i][k]=np.diag(cov2)[0]
        #plot dei dati
        plt.plot(xb,yb, '*b',markersize=7)
        plt.plot(peaks[name][0][cond3],sm.max_boltz(peaks[name][0][cond3],vmax[i],norm[i],gmu[i]),'g',linewidth=.5)
        plt.plot(peaks[name][0][cond3],sm.max_boltz(peaks[name][0][cond3],*fit),'grey',linewidth=.5)
        plt.plot(peaks[name][0][bcon],peaks[name][1][bcon],'.r')
        plt.plot(peaks[name][0][cond3],sm.max_boltz(peaks[name][0][cond3],*fit2),'k--',linewidth=1)

    plt.grid()
    plt.legend(['whole data','selected data','guess','1st fit','reconstructed band','2nd fit'])
    plt.xlabel('Frequenza [THz]')
    plt.ylabel('Assorbimento')
    plt.title(name)

Si può notare che la banda è asimmetrica, ma che in generale i dati si adattano piuttosto bene al modello scelto. Un problema che presentano questi dati però è il rumore, e ciò influenza all'ordine 0 la ricerca dei massimi, i quali costituiscono proprio i dati che utilizzo per i fit. Qui entra in gioco il primo metodo di "fit dei picchi", infatti uno dei parametri di tali fit è proprio la norma del profilo di vogit, proporzionale all'altezza del picco, e il fit agisce eliminando proprio il rumore, quindi ottengo risultati più precisi.

Come analisi finale integro quindi i risultati ottenuti dai due metodi precedenti, seleziono i dati in x grazie al fit delle bande e come dati in y utilizzo la norma dei picchi trovata grazie al fit dei picchi (ovviamente escludo dalla banda i doppi picchi di cui ho parlato in precedenza), e utilizzo sempre la funzione di Maxwell-Boltzmann. 

In [15]:
#metodo completo
plt.close('all')
print('Differenze percentuali tra paretri vmax ricavati con il fit bande semplice ed il metodo completo')
for i,name in enumerate(myList):
    #come x e y salvo la banda per intero e non divisa dx o sx
    x=np.copy(peaks[name][0][bfin[i][0]+bfin[i][1]])
    y=np.copy(fit_res[i][bfin[i][0]+bfin[i][1]])
    b_max=np.argmax(y)
    plt.figure()
    for k in range(2):
        #non ho piu` bisogno di tutti gli if poiche` sono gia` ordinati i vettori di condizioni
        xb=np.copy(peaks[name][0][bfin[i][k]])
        yb=np.copy(fit_res[i][bfin[i][k]])
        #per ovviare pero` alla mancanza degli if, utilizzo quest condizione ridotta per il plot
        cond3=np.logical_and(xb[0]<=peaks[name][0],peaks[name][0]<=xb[-1])
        #calcolo starting points
        vmax[i]=abs(xb[np.argmax(yb)]-gmu[i])
        norm[i]=np.exp(1)*yb[np.argmax(yb)]*abs(vmax[i])*(np.pi**(1/2))/4
        p0=[vmax[i],norm[i],gmu[i]]
        fit,cov=curve_fit(sm.max_boltz,xb,yb,p0=p0)
        #stamo la differenza percentuale tra i risultati del parametro vmax del fit, per evitare ripetizioni non sampo la prima volta. Uso proprio vmax perche` non dipende dalla normalizzazione
        if np.logical_or(i==1,k==1):
            print(name,k+1)
            print(abs(fit[0]-fit_mb[i][k])/fit[0])
            
        plt.plot(xb,yb, '.b',markersize=7)
        plt.plot(peaks[name][0][cond3],sm.max_boltz(peaks[name][0][cond3],*fit),'r',linewidth=.5)
    plt.grid()
    plt.legend(['selected voigt fitted band','fit'])
    plt.xlabel('Frequenza [THz]')
    plt.ylabel('Coefficienti Vogit')
    plt.title(name)

Differenze percentuali tra paretri vmax ricavati con il fit bande semplice ed il metodo completo
C2H2 2
0.0059416718679133436
CO2 1
0.004360600121019098
CO2 2
0.0017988470628379051


Si nota che la differenza percentuale tra i risultati dei due fit per vmax e` molto bassa, ma dato che vmax^2=2kT/m, nota ad esempio la massa della molecola, se volessi ricavare T, il suo valore potrebbe variare significativamente a seconda del metodo scelto. L'aspetto negativo di questo metodo, in principio più perciso del solo fit delle bande, è la ridotta quantità di dati con cui lavora.

Concludo il notebook con un paio di osservazioni
1. I risultati ottenuti sono soddisfacenti poiché i fit convergono bene e i modelli sono corretti
2. L'unione dei due metodi consente di ottenere risultati più precisi 
3. Alcune idee future, come la distinzione di bande minori o l'implementazione degli errori sono facilmente implementabili perchè altamente compatibili con i metodi già utilizzati